In [1]:
#Importing Libraries

import json
import string
import random 

import nltk
nltk.download('omw-1.4')
import numpy as np
from nltk.stem import WordNetLemmatizer 
import tensorflow as tf 
nltk.download("punkt")
nltk.download("wordnet")

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from imblearn.pipeline import Pipeline, make_pipeline
from imblearn.over_sampling import SMOTE
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn import svm 
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn import svm
from sklearn.svm import SVC

# Loading The Dataset 
df = pd.read_csv('Dataset_Cognitive_Distortions.tsv', sep='\t', header=0)
#df = df.drop(df.columns[[2, 3, 4]], axis=1)  # df.columns is zero-based pd.Index
df.shape

#Data Preprocessing 
df['Phrase'] = df['Phrase'].str.lower()


# Divide data between training and test data
X = df['Phrase']
y = df['Cognitive Distortion']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, train_size = .8)


[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


MULTINOMIAL NAIVE BAYES

In [7]:
textclassifier =Pipeline([
  ('vect', CountVectorizer()),
   ('tfidf', TfidfTransformer()),
   ('smote', SMOTE(random_state=0)),
   ('mnb', MultinomialNB())
])
params = {'smote__k_neighbors': [2,3,4,5,6,7,8,9,10, ],
          'mnb__alpha': [0.01, 0.1, 0.3, 0.5, 1.0, 10, ]}
          
#textclassifier.fit(X_train, y_train)

#y_pred = textclassifier.predict(X_test)

multinomial_nb_grid = GridSearchCV(estimator=textclassifier, param_grid=params, n_jobs=10, cv=10, verbose=5)
multinomial_nb_grid.fit(X_train, y_train)


print('Train Accuracy : %.3f'%multinomial_nb_grid.best_estimator_.score(X_train, y_train))
print('Test Accuracy : %.3f'%multinomial_nb_grid.best_estimator_.score(X_test, y_test))
print('Best Accuracy Through Grid Search : %.3f'%multinomial_nb_grid.best_score_)
print('Best Parameters : ',multinomial_nb_grid.best_params_)

y_pred = multinomial_nb_grid.predict(X_test)

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

Fitting 10 folds for each of 54 candidates, totalling 540 fits
Train Accuracy : 0.994
Test Accuracy : 0.689
Best Accuracy Through Grid Search : 0.683
Best Parameters :  {'mnb__alpha': 0.1, 'smote__k_neighbors': 10}
[[ 3  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  5  0  1  0  0  0  0  0  0  0  0  0  1  0  0]
 [ 0  0  5  0  0  0  0  1  0  0  0  0  0  0  0  0]
 [ 0  2  1  7  0  0  0  0  0  1  0  2  0  0  0  0]
 [ 0  0  0  0 11  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  2  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  3  1  0  1  0  1  0  0  0  0]
 [ 0  0  0  0  0  0  0  6  0  1  0  2  0  0  1  0]
 [ 0  0  0  0  0  0  0  0  9  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  1  5  0  0  0  0  0  0]
 [ 0  0  0  2  0  0  0  2  0  0  4  0  0  0  1  0]
 [ 0  0  0  0  0  0  0  1  0  1  1  4  1  1  2  0]
 [ 0  1  0  1  0  1  0  0  0  0  0  0  5  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  5  0  0]
 [ 0  0  0  0  0  0  0  1  1  0  0  0  1  0  3  0]
 [ 0  0  0  0  0  0 

MULTINOMIAL LOGISTIC REGRESSION

In [5]:
textclassifier =Pipeline([
  ('vect', CountVectorizer()),
   ('tfidf', TfidfTransformer()),
   ('smote', SMOTE(random_state=0)),
   ('mlg', LogisticRegression(multi_class='multinomial', random_state=0, warm_start = True,  l1_ratio = 0.5))
])

params = {'smote__k_neighbors': [2,3,4,5,6,7,8,9,10, ],
          'mlg__penalty': ['l1', 'l2', 'elasticnet', 'none' ],
          'mlg__C': [0.01, 0.1, 0.3, 0.5, 1.0,],
          'mlg__solver': ['lbfgs', 'newton-cg'
                          , 'sag', 'saga'],
          }
          
multinomial_nb_grid = GridSearchCV(estimator=textclassifier, param_grid=params, n_jobs=10, cv=10, verbose=5)
multinomial_nb_grid.fit(X_train, y_train)


print('Train Accuracy : %.3f'%multinomial_nb_grid.best_estimator_.score(X_train, y_train))
print('Test Accuracy : %.3f'%multinomial_nb_grid.best_estimator_.score(X_test, y_test))
print('Best Accuracy Through Grid Search : %.3f'%multinomial_nb_grid.best_score_)
print('Best Parameters : ',multinomial_nb_grid.best_params_)

y_pred = multinomial_nb_grid.predict(X_test)

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

Fitting 10 folds for each of 720 candidates, totalling 7200 fits


c:\Users\Usuario\Desktop\Kai-SelfCare-Chatbot\.venv\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
2700 fits failed out of a total of 7200.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
450 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\Usuario\Desktop\Kai-SelfCare-Chatbot\.venv\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\Usuario\Desktop\Kai-SelfCare-Chatbot\.venv\lib\site-packages\imblearn\pipeline.py", line 297, in fit
    self._final_estimator.fit(Xt, yt, **fit_params_last_step)
  File "c:\Users\Usuario\Desktop\Kai-SelfCare-Chatbot\.venv\lib\site-pa

Train Accuracy : 0.998
Test Accuracy : 0.647
Best Accuracy Through Grid Search : 0.687
Best Parameters :  {'mlg__C': 0.01, 'mlg__penalty': 'none', 'mlg__solver': 'saga', 'smote__k_neighbors': 2}
[[ 3  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  3  0  1  0  0  2  0  0  0  0  0  0  1  0  0]
 [ 0  0  5  0  0  0  0  0  0  1  0  0  0  0  0  0]
 [ 0  2  1  7  0  0  0  0  0  1  0  2  0  0  0  0]
 [ 0  0  0  0 11  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  2  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  1  1  0  2  0  1  1  0  0  0]
 [ 0  0  0  0  0  0  0  4  0  1  0  4  0  0  1  0]
 [ 0  0  0  0  0  0  0  0  9  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  1  0  5  0  0  0  0  0  0]
 [ 0  0  0  1  1  0  0  0  0  0  5  0  1  0  1  0]
 [ 0  0  0  0  0  0  1  1  0  1  1  4  1  1  1  0]
 [ 1  0  0  0  0  0  0  0  0  0  1  0  6  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  5  0  0]
 [ 0  0  1  1  0  0  0  1  0  0  0  0  1  0  2  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0

c:\Users\Usuario\Desktop\Kai-SelfCare-Chatbot\.venv\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


SVM

In [5]:
textclassifier =Pipeline([
  ('vect', CountVectorizer()),
   ('tfidf', TfidfTransformer()),
   ('smote', SMOTE(random_state=0)),
   ('svm', svm.SVC(probability = True))
])

params = {'smote__k_neighbors': [2,3,4,5,6,7,8,9,10,],
        'svm__decision_function_shape': ['ovo', 'ovr']}
          
#textclassifier.fit(X_train, y_train)

#y_pred = textclassifier.predict(X_test)

multinomial_nb_grid = GridSearchCV(estimator=textclassifier, param_grid=params, n_jobs=10, cv=10, verbose=5)
multinomial_nb_grid.fit(X_train, y_train)


print('Train Accuracy : %.3f'%multinomial_nb_grid.best_estimator_.score(X_train, y_train))
print('Test Accuracy : %.3f'%multinomial_nb_grid.best_estimator_.score(X_test, y_test))
print('Best Accuracy Through Grid Search : %.3f'%multinomial_nb_grid.best_score_)
print('Best Parameters : ',multinomial_nb_grid.best_params_)

y_pred = multinomial_nb_grid.predict(X_test)

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

Fitting 10 folds for each of 22 candidates, totalling 220 fits


c:\Users\Usuario\Desktop\Kai-SelfCare-Chatbot\.venv\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
20 fits failed out of a total of 220.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
6 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\Usuario\Desktop\Kai-SelfCare-Chatbot\.venv\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\Usuario\Desktop\Kai-SelfCare-Chatbot\.venv\lib\site-packages\imblearn\pipeline.py", line 293, in fit
    Xt, yt = self._fit(X, y, **fit_params_steps)
  File "c:\Users\Usuario\Desktop\Kai-SelfCare-Chatbot\.venv\lib\site-packages\imblearn\pi

Train Accuracy : 0.998
Test Accuracy : 0.672
Best Accuracy Through Grid Search : 0.637
Best Parameters :  {'smote__k_neighbors': 10, 'svm__decision_function_shape': 'ovo'}
[[ 2  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0]
 [ 0  3  0  1  0  0  0  0  0  0  0  2  0  1  0  0]
 [ 0  0  4  0  0  0  0  0  0  0  0  1  1  0  0  0]
 [ 0  0  0  7  0  0  0  0  0  0  0  6  0  0  0  0]
 [ 0  0  0  0 11  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  1  0  0  0  0  1  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  1  0  1  0  2  1  0  0  1]
 [ 0  0  0  0  0  0  0  4  0  0  0  5  0  0  1  0]
 [ 0  0  0  0  0  0  0  0  9  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  1  0  5  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  6  2  0  0  1  0]
 [ 0  0  0  0  0  0  0  0  0  0  0 11  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  2  6  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  1  0  4  0  0]
 [ 0  0  0  1  0  0  0  0  0  0  0  3  0  0  2  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  2  5]]
           

c:\Users\Usuario\Desktop\Kai-SelfCare-Chatbot\.venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Usuario\Desktop\Kai-SelfCare-Chatbot\.venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Usuario\Desktop\Kai-SelfCare-Chatbot\.venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier

RANDOM FORESTS

In [22]:
textclassifier =Pipeline([
  ('vect', CountVectorizer()),
   ('tfidf', TfidfTransformer()),
   ('smote', SMOTE(random_state=0)),
   ('rf', RandomForestClassifier(random_state=0))
])

params = {'smote__k_neighbors': [2,3,4,5,6,7,8,9,10, ]}
          
#textclassifier.fit(X_train, y_train)

#y_pred = textclassifier.predict(X_test)

multinomial_nb_grid = GridSearchCV(estimator=textclassifier, param_grid=params, n_jobs=10, cv=10, verbose=5)
multinomial_nb_grid.fit(X_train, y_train)


print('Train Accuracy : %.3f'%multinomial_nb_grid.best_estimator_.score(X_train, y_train))
print('Test Accuracy : %.3f'%multinomial_nb_grid.best_estimator_.score(X_test, y_test))
print('Best Accuracy Through Grid Search : %.3f'%multinomial_nb_grid.best_score_)
print('Best Parameters : ',multinomial_nb_grid.best_params_)

y_pred = multinomial_nb_grid.predict(X_test)

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

Fitting 10 folds for each of 9 candidates, totalling 90 fits
Train Accuracy : 0.998
Test Accuracy : 0.689
Best Accuracy Through Grid Search : 0.668
Best Parameters :  {'smote__k_neighbors': 3}
[[2 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 5 0 1 0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 4 0 0 0 0 0 1 0 0 0 0 0 1 0]
 [0 2 0 6 1 0 0 0 0 0 0 2 0 0 0 2]
 [0 0 0 0 9 0 0 0 0 0 0 0 2 0 0 0]
 [0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 1 0 1 0 1 1 0 0 1]
 [0 0 0 0 0 0 0 5 0 0 0 3 1 0 0 1]
 [0 0 0 0 0 0 0 0 9 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 2 0 3 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 7 0 0 0 1 1]
 [0 0 1 0 0 0 0 1 0 0 0 9 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 1 5 0 0 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 5 0 0]
 [0 0 0 1 0 0 0 0 1 0 0 1 0 0 3 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 7]]
                         precision    recall  f1-score   support

     Always Being Right       1.00      0.67      0.80         3
                Blaming       0.62      0.71      0.67         7
        Catastrophizing       0.80      0.67      

LightGBM

In [23]:
textclassifier =Pipeline([
  ('vect', CountVectorizer()),
   ('tfidf', TfidfTransformer()),
   ('smote', SMOTE(random_state=0)),
   ('lgbm', LGBMClassifier(random_state=0))
])

params = {'smote__k_neighbors': [2,3,4,5,6,7,8,9,10, ]}
          
#textclassifier.fit(X_train, y_train)

#y_pred = textclassifier.predict(X_test)

multinomial_nb_grid = GridSearchCV(estimator=textclassifier, param_grid=params, n_jobs=10, cv=10, verbose=5)
multinomial_nb_grid.fit(X_train, y_train)


print('Train Accuracy : %.3f'%multinomial_nb_grid.best_estimator_.score(X_train, y_train))
print('Test Accuracy : %.3f'%multinomial_nb_grid.best_estimator_.score(X_test, y_test))
print('Best Accuracy Through Grid Search : %.3f'%multinomial_nb_grid.best_score_)
print('Best Parameters : ',multinomial_nb_grid.best_params_)

y_pred = multinomial_nb_grid.predict(X_test)

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

Fitting 10 folds for each of 9 candidates, totalling 90 fits
Train Accuracy : 0.996
Test Accuracy : 0.605
Best Accuracy Through Grid Search : 0.595
Best Parameters :  {'smote__k_neighbors': 10}
[[3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 3 0 0 0 0 0 0 0 0 0 0 4 0 0 0]
 [0 0 4 0 0 0 0 0 0 1 0 0 0 0 1 0]
 [0 0 0 5 2 0 0 0 0 0 0 3 1 0 0 2]
 [0 0 0 0 6 0 0 0 0 0 0 0 3 0 0 2]
 [0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0]
 [0 0 1 0 0 0 1 0 0 0 0 1 1 0 0 2]
 [0 1 0 1 0 1 0 4 0 1 0 1 1 0 0 0]
 [0 0 0 0 0 0 0 0 8 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 2 0 4 0 0 0 0 0 0]
 [0 0 0 1 1 0 0 0 0 0 7 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 2 7 1 0 0 0]
 [0 0 0 0 0 1 0 1 0 0 0 0 5 0 0 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 5 0 0]
 [0 0 1 1 0 0 0 0 1 0 1 0 0 0 2 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 7]]
                         precision    recall  f1-score   support

     Always Being Right       1.00      1.00      1.00         3
                Blaming       0.75      0.43      0.55         7
        Catastrophizing       0.67      0.67     

KNN

In [24]:
textclassifier =Pipeline([
  ('vect', CountVectorizer()),
   ('tfidf', TfidfTransformer()),
   ('smote', SMOTE(random_state=0)),
   ('knn', KNeighborsClassifier())
])

params = {'knn__n_neighbors': [5,6,7,8,9,10, ],
          'knn__weights': ['uniform', 'distance'],
          'smote__k_neighbors': [2,3,4,5,6,7,8,9,10, ]
          }
          
#textclassifier.fit(X_train, y_train)

#y_pred = textclassifier.predict(X_test)

multinomial_nb_grid = GridSearchCV(estimator=textclassifier, param_grid=params, n_jobs=10, cv=10, verbose=5)
multinomial_nb_grid.fit(X_train, y_train)


print('Train Accuracy : %.3f'%multinomial_nb_grid.best_estimator_.score(X_train, y_train))
print('Test Accuracy : %.3f'%multinomial_nb_grid.best_estimator_.score(X_test, y_test))
print('Best Accuracy Through Grid Search : %.3f'%multinomial_nb_grid.best_score_)
print('Best Parameters : ',multinomial_nb_grid.best_params_)

Fitting 10 folds for each of 108 candidates, totalling 1080 fits
Train Accuracy : 0.998
Test Accuracy : 0.529
Best Accuracy Through Grid Search : 0.540
Best Parameters :  {'knn__n_neighbors': 5, 'knn__weights': 'distance', 'smote__k_neighbors': 10}


In [25]:
y_pred = multinomial_nb_grid.predict(X_test)

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

[[3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 3 0 1 0 0 2 0 0 0 0 0 0 1 0 0]
 [0 0 2 2 0 0 0 0 0 0 0 0 1 1 0 0]
 [1 3 0 7 0 0 1 0 0 0 0 1 0 0 0 0]
 [2 0 0 0 7 0 0 0 0 0 0 0 2 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0]
 [1 0 0 0 0 0 3 0 0 0 0 0 1 1 0 0]
 [0 0 1 0 0 0 0 7 0 1 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 9 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 1 4 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 2 0 0 3 0 0 1 3 0]
 [1 0 0 0 0 1 1 3 0 1 0 0 0 2 2 0]
 [1 0 1 0 0 0 0 0 0 0 1 0 4 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 5 0 0]
 [0 0 0 1 0 1 0 0 1 0 0 0 0 1 2 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 1 0 2 3]]
                         precision    recall  f1-score   support

     Always Being Right       0.33      1.00      0.50         3
                Blaming       0.50      0.43      0.46         7
        Catastrophizing       0.50      0.33      0.40         6
      Control Fallacies       0.64      0.54      0.58        13
    Emotional Reasoning       1.00      0.64      0.78        11
      Fallacy Of Change       0.33      0.50    